In [1]:
from enum import Enum

class AgentMode(Enum):
    DIAGNOSTIC = "Diagnostic"  # Diagnostic mode for medical inquiry (問診模式")
    TONGUE_DIAGNOSIS = "TongueDiagnosis"  # Tongue diagnosis mode for health assessment (舌診模式)
    EVALUATION_ADVICE = "EvaluationAndAdvice"  # Evaluation and advice mode (評估和建議模式)
    INQUIRY = "Inquiry"    # Inquiry mode for general questions (自由提問模式)
    CHITCHAT = "Chitchat"      # Chitchat mode for casual conversation (閒聊模式)

In [4]:
import os
from langchain_community.llms import Ollama
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.embeddings import OllamaEmbeddings 
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

model_name = "llamafamily/llama3-chinese-8b-instruct"
file_path = "C:\\Users\\User\\Documents\\work\\tcm-agent\\data\\txt"

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use five sentences maximum and keep the "
    "answer concise. Answer in Chinese."
    "\n\n"
    "{context}"
)

llm = Ollama(model=model_name,temperature=0.2,top_p=0.3)
embedding= OllamaEmbeddings(model=model_name,temperature=0.2,top_p=0.3)
# loader = TextLoader("C:\\Users\\User\\Downloads\\中醫體質與問診.txt")
# loader = TextLoader("C:\\Users\\User\\Downloads\\九大體質.txt")


class RagChain:
    def __init__(self,llm, embedding,file_name,system_prompt) -> None:
        self.llm = llm
        self.embedding = embedding
        # self.system_prompt = system_prompt
        
        total_path = os.path.normpath(f"{file_path}/{file_name}")
        self.loader = TextLoader(total_path,encoding='utf-8')
        self.docs = self.loader.load()
        
        self.text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        self.splits = self.text_splitter.split_documents(self.docs)
        self.vector_store = Chroma.from_documents(documents= self.splits, embedding= embedding)
        self.retriever = self.vector_store.as_retriever()
        
        self.prompt = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt),
                ("human", "{input}"),
            ]
        )
        
        self.question_answer_chain = create_stuff_documents_chain(self.llm, self.prompt)
        self.rag_chain = create_retrieval_chain(self.retriever, self.question_answer_chain)
            
    def __call__(self, input_data):
        return self.rag_chain.invoke(input_data)

    
# 中醫十問
ZhongyiShiwen = RagChain(llm, embedding, "中醫十問.txt", system_prompt)   
# 九大體質
JiudaTizhi = RagChain(llm, embedding, "九大體質.txt", system_prompt) 
SelfIntroduction = RagChain(llm, embedding, "自我介紹.txt", system_prompt)  
SongBieYu = RagChain(llm, embedding, "送別語.txt", system_prompt)  



# response_1 = ZhongyiShiwen({"input": "based on knowledge。請按照＂自我介紹.txt＂文件做自我介紹"})
# print(response_1['answer'])

# response_2 = JiudaTizhi({"input": "based on knowledge。請分析文件，中醫十問是哪十問?請告訴我問診名稱就好。"})
# print(response_2['answer'])

# response_3 = SelfIntroduction({"input": "based on knowledge。請分析文件中有提到以下描述嗎?如果沒有請回答不知道，如果有，這是哪一種體質?\n- - -\n常見表現特徵為手腳冰冷、臉色蒼白、身體畏寒、喜歡吃溫熱飲食。容易發生水腫、腹瀉大便不成形，易表現出精神不濟，容易感冒。體態上呈現肌肉鬆軟不結實，舌頭胖大、舌苔變厚呈白色水滑。"})
# print(response_3['answer'])


# response_4 = SongBieYu({"input": "based on knowledge。請參考＂送別語.txt＂文件做送別的話語"})
# print(response_4['answer'])

In [22]:

from abc import ABC, abstractmethod

# redis.core = RedisCore()

class InteractionStrategy(ABC):
    @abstractmethod
    def handle_interaction(self, agent, user_input):
        pass

      
# Diagnostic mode for medical inquiry (問診模式")
class DiagnosticStrategy(InteractionStrategy):
    def handle_interaction(self, agent, user_input):
        response = None
        if agent.question_count == 0:
            self_introduction_response = SelfIntroduction({"input": "根據知識庫。請按照＂自我介紹.txt＂文件做自我介紹"})
            zhongyi_shiwen_response = ZhongyiShiwen({"input": "根據知識庫。中醫十問當中的第一問的問診細節是什麼?只要問診細節：後面的文字就好"})
            question_key = agent.qa[agent.question_count]
            agent.questions[question_key] = zhongyi_shiwen_response['answer']

            response = llm.invoke(f"請把以下兩句話請按照順序進行合併。第一句：{self_introduction_response['answer']}\n 接著讓我們來開始第一個問題 \n第二句：{zhongyi_shiwen_response['answer']}")
        elif (agent.sex == 'male' and agent.question_count >= 9) or (agent.sex == 'female' and agent.question_count >= 10):
            # 源邏輯，移去評估和建議了
            # response = llm.invoke(f"請把以下幾段話請按照順序進行合併。\n第一句話: 接著讓我們進行舌診，請將舌頭伸出，配合畫面上的資訊 \n　第二句話: 我們會結合剛剛的問診和舌診給出評估和建議")
            # response = f"接下是舌診，請將舌頭伸出，配合畫面上的指示。我們會結合剛剛的問診和舌診給出評估和建議"
            # RedisCore.publisher(RedisChannel.do_aikanshe_service,'do ai kan she from Agent strategy.')
            agent.mode = AgentMode.TONGUE_DIAGNOSIS 
            agent.invoke('接著是舌診')
            # response=
            
        else:
            answer_key = agent.qa[agent.question_count - 1]
            agent.answers[answer_key] = user_input

            key = agent.qa[agent.question_count]
            zhongyi_shiwen_response = ZhongyiShiwen({"input": f"根據知識庫。中醫十問當中的{key}的問診細節是什麼?只要問診細節：後面的的文字就好"})

            question_key = agent.qa[agent.question_count]
            agent.questions[question_key] = zhongyi_shiwen_response['answer']

            llm_response = llm.invoke(f"如果以下文字內容是正面的，請你回答＂好的，我知道了！＂；如果以下文字內容是負面的，請總結以下文字大概意思就好字數不超過二十字，並在最前面加上＂我了解你有(_總結後的文字_)的問題。＂。\n{user_input}")
            # response = llm.invoke(f"請把以下幾段話請按照順序進行合併。 {llm_response} \n　接著讓我們來進行下一個問題 \n {zhongyi_shiwen_response['answer']}")
            response = f"{llm_response} 。接著讓我們來進行下一個問題，{zhongyi_shiwen_response['answer']}"

        agent.question_count += 1
        if response is not None:
            agent.messages.append(response)
        return response

# Tongue diagnosis mode for health assessment (舌診模式)
class TongueDiagnosis(InteractionStrategy):
    def handle_interaction(self, agent, user_input):
        print(user_input)
        # agent.redis_client.publish(R)
        # RedisCore.publisher(RedisChannel.do_aikanshe_service,'do ai kan she from Agent strategy.')
        # response = llm.invoke(f"Chitchat response for input: {user_input}")
        response = f"接下是舌診，請將舌頭伸出，配合畫面上的指示。我們會結合剛剛的問診和舌診給出評估和建議"
        # RedisCore.publisher(RedisChannel.do_aikanshe_service,'do ai kan she from Agent strategy.')
        agent.messages.append(response)
        return response
        # pass

# Evaluation and advice mode (評估和建議模式)
class EvaluationAndAdvice(InteractionStrategy):
    def handle_interaction(self, agent, user_input):
        formatted_list = [f"問{key}，答 {value}" for key, value in agent.answers.items()]
        formatted_string = "。".join(formatted_list)
        jiuda_tizhi_response = JiudaTizhi({"input": f"根據知識庫。請分析文件，以下描述最接近哪幾種體質特徵(1~3種)?有什麼飲食上的建議?請用中文回答\n- - -\n{formatted_string}"})
        response = jiuda_tizhi_response['answer']
        agent.messages.append(response)
        return response

# Inquiry mode for general questions (自由提問模式)
class InquiryStrategy(InteractionStrategy):
    def handle_interaction(self, agent, user_input):
        # 自由提問模式的具體邏輯在這裡實現
        # 串接 RagFlow 使用大量額外資料，提供自由問答
        response = llm.invoke(f"Chitchat response for input: {user_input}")
        agent.messages.append(response)
        return response
# Chitchat mode for casual conversation (閒聊模式)
class ChitchatStrategy(InteractionStrategy):
    def handle_interaction(self, agent, user_input):
        # 閒聊模式的具體邏輯在這裡實現
        response = llm.invoke(f"Chitchat response for input: {user_input}")
        agent.messages.append(response)
        return response

In [5]:

# 获取项目根目录
import os
import sys
from typing import Literal

from reactivex.subject.behaviorsubject import BehaviorSubject
from redis import Redis


class Agent:
    user_info_from_yolo = None
    modeSub:BehaviorSubject
    def __init__(self):
        self.__mode = AgentMode.DIAGNOSTIC
        self.modeSub = BehaviorSubject(self.__mode)
        self.__qa = ["睡眠", "胃口", "大便", "小便", "口渴", "寒熱", "汗", "體力", "性功能", "女子月經"]
        self.initial()
        

    def initial(self):
        self.user_info_from_yolo = None
        self.__sex: Literal['male', 'female']  = 'male'
        self.__messages = []
        self.__questions = {}
        self.__answers = {}
        self.__question_count = 0
        self.__strategy = DiagnosticStrategy()
        self.transformed_data = {"age": None, "male": None}
        # self.question_count = None
        
    @property
    def mode(self):
        return self.__mode
    @property
    def sex(self) -> Literal['male', 'female']:
        return self.__sex
    @property
    def messages(self):
        return self.__messages
    @property
    def qa(self):
        return self.__qa
    @property
    def questions(self):
        return self.__questions
    @property
    def answers(self):
        return self.__answers
    @property
    def question_count(self):
        return self.__question_count
    
    @mode.setter
    def mode(self, value):
        if self.__mode != value:
            self.modeSub.on_next(value)
            
        self.__mode = value
        if value == AgentMode.DIAGNOSTIC:
            self.__strategy = DiagnosticStrategy()
        elif value == AgentMode.TONGUE_DIAGNOSIS:
            self.__strategy = TongueDiagnosis()
        elif value == AgentMode.EVALUATION_ADVICE:
            self.__strategy = EvaluationAndAdvice()
        elif value == AgentMode.INQUIRY:
            self.__strategy = InquiryStrategy()
        elif value == AgentMode.CHITCHAT:
            self.__strategy = ChitchatStrategy()
            
    @sex.setter
    def sex(self, value: Literal['male', 'female']):
        if value not in ['male', 'female']:
            raise ValueError("Sex must be 'male' or 'female'")
        self.__sex = value
    @question_count.setter
    def question_count(self, value):
        if value is None:
            pass
        elif value < self.__question_count:
            print(f"請注意，此操作可能會導致重複回答已經回答過的問題。\n目前問題是關於:(問題{self.__question_count}){self.__qa[self.__question_count]}，你將回到(問題{value}){self.__qa[value]}。")
        elif value > self.__question_count and value != self.__question_count + 1:
            print(f"請注意，此操作可能會導致問題被跳過。\n目前問題是關於:(問題{self.__question_count}){self.__qa[self.__question_count]}，你將跳到(問題{value}){self.__qa[value]}。")
        self.__question_count = value

    def invoke(self, user_input):
        if self.question_count is not None:
            return self.__strategy.handle_interaction(self, user_input)
        else:
            return '請稍後在試。'

In [23]:
agent = Agent()

In [24]:
agent.invoke('開始問診')

'您好，我是一名中醫師，在治療過程中，可能會出現一些反應，這是正常的，不需要擔心。請記住，中醫治病是根據身體的症狀，而非西醫的檢驗報告，所以請詳細描述您的症狀，以便我們能夠為您提供最好的治療。讓我們來開始第一個問題，您好嗎？'

In [25]:
agent.question_count=9

請注意，此操作可能會導致問題被跳過。
目前問題是關於:(問題1)胃口，你將跳到(問題8)性功能。


In [27]:
agent.invoke('性功能一切正常')

接著是舌診


In [28]:
agent.messages

['您好，我是一名中醫師，在治療過程中，可能會出現一些反應，這是正常的，不需要擔心。請記住，中醫治病是根據身體的症狀，而非西醫的檢驗報告，所以請詳細描述您的症狀，以便我們能夠為您提供最好的治療。讓我們來開始第一個問題，您好嗎？',
 '我了解你有正常性的問題。 。接著讓我們來進行下一個問題，問診問題： 性功能\n問診細節： 你性功能好嗎？等等。',
 '接下是舌診，請將舌頭伸出，配合畫面上的指示。我們會結合剛剛的問診和舌診給出評估和建議']

In [29]:
agent.mode

<AgentMode.TONGUE_DIAGNOSIS: 'TongueDiagnosis'>